In [1]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(
                  os.path.dirname(sys.path[0]), 
                  os.pardir)
)
sys.path.append(PROJECT_ROOT)

import numpy as np
from matplotlib import pyplot as plt

import evosquares.perturbations
from evosquares.operators.crossover import *
from evosquares.operators.mutation import *

from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.dnsga2 import DNSGA2
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.operators.crossover.ux import UniformCrossover
from pymoo.core.crossover import Crossover
from pymoo.core.duplicate import ElementwiseDuplicateElimination
from pymoo.core.mutation import Mutation
from pymoo.operators.selection.rnd import RandomSelection
from pymoo.core.sampling import Sampling

import scipy
from functools import partial
import multiprocessing as mp

ModuleNotFoundError: No module named 'operators'

In [ ]:
class MatchSwapCrossover(Crossover):
    def __init__(self, n_closest, repair_steps=0, repair_size=0, **kwargs):
        # define the crossover: number of parents and number of offsprings
        self.n_closest = n_closest
        self.repair_steps = repair_steps
        self.repair_size = repair_size
        super().__init__(2, 2, **kwargs)

    def _do(self, problem, X, **kwargs):
        return match_swap_crossover(X, self.n_closest, self.repair_steps, self.repair_size)

In [ ]:
class RandomSwapCrossover(Crossover):
    def __init__(self, n_closest, repair_steps=0, repair_size=0, **kwargs):
        # define the crossover: number of parents and number of offsprings
        self.n_closest = n_closest
        self.repair_steps = repair_steps
        self.repair_size = repair_size
        super().__init__(2, 2, **kwargs)

    def _do(self, problem, X, **kwargs):
        return random_swap_crossover(X, self.n_closest, self.repair_steps, self.repair_size)

In [ ]:
class SquareDuplicateElim(ElementwiseDuplicateElimination):
    def is_equal(self, a, b):
        return np.array_equal(a, b)

In [ ]:
class CombinedMutation(Mutation):
    def __init__(self, mutation_list):
        self.mutation_list = mutation_list
        super().__init__()

    def _do(self, problem, X, **kwargs):
        for mutation_op in self.mutation_list:
            X = mutation_op(X)
        
        return X

In [ ]:
class PerturbSampling(Sampling):
    def __init__(self, n, **kwargs):
        self.n = n
        self.num = np.ceil(np.sqrt(self.n)).astype('int')
        super().__init__(**kwargs)
        
    def _do(self, problem, n_samples, **kwargs):
        num = self.num
        size = 2 / num
        
        ls = np.linspace(start=-1 + size/2, stop=1 - size/2, num=num)
        np.random.shuffle(ls)
        
        inds = np.random.permutation(num*num)[:self.n]
        x = np.tile(np.expand_dims(np.tile(ls, num), 0), (n_samples, 1))[:, inds]
        y = np.tile(np.expand_dims(np.repeat(ls, num), 0), (n_samples, 1))[:, inds]
        t = np.random.random(size=x.shape) * np.pi/2
        new_x, new_y, new_t = perturbations.uniform_shift_square(x, y, t, size/20)
        X = np.concatenate((new_x, new_y, new_t), axis=1)
        
        return X

In [ ]:
ns = 29
samp = PerturbSampling(ns)
gen = samp._do(None, 1)
print(gen.shape)
perturbations.draw_squares(gen[0])

In [ ]:
# definition of square packing problem
class SquarePacking(Problem):
    def __init__(self, n):
        self.n = n        

        # representation is array of size 3n
        # first n is X coord of centers, then Y coords, then angles
        xl = np.concatenate((np.full(n, fill_value= -1), np.full(n, fill_value= -1), np.full(n, fill_value=0)))
        xu = np.concatenate((np.full(n, fill_value= 1), np.full(n, fill_value= 1), np.full(n, fill_value=np.pi/2)))
        
        super().__init__(n_var=3*n, n_obj=1, xl=xl, xu=xu, n_constr=0)
        
    def _evaluate(self, x, out, *args, **kwargs):
        bound_size = perturbations.find_bound(x)
        angles = x[:, -self.n:]
        #angle_variance = scipy.stats.circvar((x[:, -self.n:]), high=np.pi/2, low=0, axis=1)
        #angle_dist1 = np.mean(np.abs((x[:, -self.n:]) - np.pi/4), axis=1)        
        #angle_dist2 = np.mean(np.minimum(np.abs((x[:, -self.n:]) - np.pi/2), np.abs((x[:, -self.n:]))), axis=1)
        out["F"] = [bound_size]
            

In [ ]:
def experiment(proc_num, return_dict, n, seed):
    mag = 2 / np.ceil(np.sqrt(n))
    problem = SquarePacking(n=n)

    # define mutations
    correction_mutation = partial(random_walk_mutation, walk_prob=0.5, walk_steps=(n*8,), walk_size=(mag/16,), no_theta_prob=0.8)
    exploration_mutation = partial(perturb_mutation, perturb_prob=0.2, perturb_num=int(n/2), perturb_size=mag*2)
    rot_mutation = partial(rotation_mutation, rot_prob=0.2, rot_max_num=int(n/4), rot_size=np.pi/4)

    mutation = CombinedMutation([rot_mutation, exploration_mutation, correction_mutation])

    # define crossover
    crossover= RandomSwapCrossover(prob=0.5, n_closest=n)

    #algorithm = NSGA2(pop_size=100, crossover=crossover, mutation=mutation, eliminate_duplicates=None, sampling=PerturbSampling(n))
    algorithm = GA(pop_size=100, crossover=crossover, mutation=mutation, eliminate_duplicates=None)
    res = minimize(problem, algorithm, ('n_gen', 1000), seed=seed, verbose=proc_num == 0)
    
    return_dict[proc_num] = res

In [ ]:
n = 29
mag = 2 / np.ceil(np.sqrt(n))

# p1, p2, p3
arg_list = [(np.random.randint(100), ) for _ in range(8)]

manager = mp.Manager()
return_dict = manager.dict()

jobs = []
for i, args in enumerate(arg_list):
    p = mp.Process(target=experiment, args=(i, return_dict, n, *args))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

In [ ]:
# find the solution with num_overlapping overlapping squares with the smallest bounding box
vals = return_dict.values()
vals = [(data.X, data.F) for data in vals]
print(min(vals, key=lambda x: x[1])[1])
print(np.mean([d[1] for d in vals]))
for i, args in enumerate(arg_list):
    print(args, return_dict[i].F)
    perturbations.draw_squares(return_dict[i].X)

In [ ]:
# find the solution with num_overlapping overlapping squares with the smallest bounding box
num_overlapping = 0
print(res.X.shape)
best_x = res.X[0]
best_size = res.F[0][0]
sorted_XF = sorted(zip(res.X, res.F), key=lambda p: p[1][0])

fig, axs = plt.subplots(3, 3)
fig.set_size_inches(7, 7)
for i, ax in enumerate(axs.flatten()):
    perturbations.subplot_squares(ax, sorted_XF[i][0])
    ax.set_title(sorted_XF[i][1][0])
    ax.axis('off')

In [ ]:
fine_tuned_best = perturbations.with_perturbations(best_x, 0.1, 10e-8, 1.5, 500)
print(np.sqrt(2) / perturbations.find_max_size(fine_tuned_best))
perturbations.draw_squares(fine_tuned_best)